In [37]:
import networkExpansionPy.lib as ne
import numpy as np
import pandas as pd
import pickle as pickle
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import spearmanr,pearsonr,mannwhitneyu
from statsmodels.stats.multitest import multipletests
plt.rcParams['font.family'] = 'Arial'
import pingouin as pg
import re


import warnings
from scipy.sparse import (spdiags, SparseEfficiencyWarning, csc_matrix,
    csr_matrix, isspmatrix, dok_matrix, lil_matrix, bsr_matrix)
warnings.simplefilter('ignore',SparseEfficiencyWarning)

def load_data(network_name):
    with open(ne.asset_path  + "/metabolic_networks/" + network_name,"rb") as filehandler:
        metabolism = pickle.load(filehandler)
    return metabolism

# load the metabolism object from the repo
model_id = "metabolism.v8.01May2023.pkl"
metabolism = pd.read_pickle('/Users/joshuagoldford/Documents/github/network-expansion-assets/models/' + model_id)
cids = metabolism.coenzymes


seed = pd.read_csv('../assets/seed_set.csv')
#background = seed[seed["Type"].isin(["Background"])].ID.tolist()
#cpd_base = seed[~seed["Type"].isin(['Sulfur source',"Nitrogen source"])].ID.tolist()
carbon_compounds = seed[~seed["Type"].isin(["Organic carbon"])].ID.tolist()

def get_c_num(formula):
    match = re.search('C(\d+)', formula)
    if match:
        return  match.group(1)
    else:
        return 0

# get only the compounds with CHO in the formula
DOR = pd.read_csv("../assets/keggCHOmolecules.reductionState.csv",index_col=0)
DOR["c"]= DOR.formula.apply(lambda x: get_c_num(x))
DOR = DOR[~DOR.c.apply(lambda x: x is None)]
DOR["c"] = DOR.c.apply(int)

In [38]:
import json
with open(ne.asset_path + "/KEGG/compound.json") as jsonfile:
    compounds = json.load(jsonfile)


In [39]:
# 
import json
import re

with open(ne.asset_path + "/KEGG/compound.json") as jsonfile:
    compounds = json.load(jsonfile)

def get_c_num(formula):
    match = re.search('C(\d+)', formula)
    if match:
        return  match.group(1)
    else:
        return 0
    
num_carbs = {"cid":[],"num_carbons":[]}
for c in compounds:
    num_carbons = get_c_num(compounds[c]["formula"])
    num_carbs["cid"].append(c)
    num_carbs["num_carbons"].append(float(num_carbons))
    
num_carbs = pd.DataFrame(num_carbs)
carbon_containing_molecules = num_carbs[num_carbs["num_carbons"] < 7]
carbon_containing_molecules = carbon_containing_molecules[carbon_containing_molecules.num_carbons > 0]


In [40]:
carbon_containing_molecules = num_carbs[num_carbs["num_carbons"] < 7]
carbon_containing_molecules = carbon_containing_molecules[carbon_containing_molecules.num_carbons > 0]

In [41]:
carbon_containing_molecules = carbon_containing_molecules[carbon_containing_molecules.cid.isin(metabolism.network.cid.unique().tolist())]

In [42]:
carbon_containing_molecules

,cid,num_carbons
11,C00012,2.0
16,C00017,2.0
21,C00022,3.0
24,C00025,5.0
25,C00026,5.0
...,...,...
18795,C22360,6.0
18797,C22362,2.0
18799,C22364,3.0
18801,C22367,5.0
